# **Introduction**

Welcome to the world of Molecular Dynamics! This lesson is to teach you the basic protocol of analyzing a molecular dynamics (MD) simulation, including a bit of Python coding. If you don't know how to code, don't worry! These lessons assume no prior knowledge of code or Python.

If you have not completed Step 1 and would like to do so, click this link: https://colab.research.google.com/drive/1vKxP4MJhdIODzScluSeU73NHOWwJIPn8?usp=sharing

## Analyzing molecular dynamics
MD simulates the motion of molecules and can describe how the structure changes over time. The analysis of molecular dynamics extracts properties from the "movie" of the molecule such as how mobile areas of the molecule were and how the pKa values of certain functional groups can vary as the molecule moves. This information can help describe the mechanism of action as conformational changes in enzymes are linked to catalytic activity and pKa is related to nucleophilicty.

## What will I get from this Colab?
The first steps of the colab show whether the molecule reached equilibrium during the simulation, which indicates how reliable or confidently you can interpret the rest of the values. The second section of the Colab describes which areas of the molecule are most dynamic. Finally, the code will calculate the pKa values for each frame of the simulation to show pKa values over time. These values and outputs can be compared to simulations of mutants or the protein with ligands to determine how these changes to the simulation environment alter the dynamics of the molecule.

[Guide video for using this colab](https://www.loom.com/share/cb783e87ba294bee916d0fd1e0847fee?sid=ac7a0852-00c7-4cc9-b2b1-cd142ee9b7fc)



---

This document has a list of [Frequently Asked Questions](https://docs.google.com/document/d/1kzkOi1T6QYjcyIoMXpU114B15WE0VcGnof5xvG5zdvA/edit?usp=sharing), reach out to your instructor if you have additional questions.


####**Please make a copy of this colab for your personal use!!**

A few things to start:

1.   These lessons only work in Google Chrome
2.   If you want to save your progress, go to File> Save a Copy in Drive; then locate a spot in your Drive folder
3.   Clicking the "play" button to the top left of a code block runs the code. Sometimes you can see the code and interact with it. However, if the code is hidden  it is to run adminstrative tasks in the background and you do not need to worry unless you are interested.
4. Sometimes the code may be hidden from view. To view it, click the '>' on the left of the title, until it changes to 'v'. This will reveal the code in that line.

Here are the files you will need for this code to run:
*   **.dcd** is a file format that store simulation trajectories. Simulation trajectories are a dynamic view of various properties in your protein, including atom coordinates, box size, velocities, and forces. Trajectory files looks at these variables as they change throughout time. You should have this file from your MD run in step 1.
*  A **.pdb** file containing a set of atomic coordinates of your molecule. **.pdb** stands for a Protein Data Bank file. It is a file containing the three-dimensional structural data of your protein directly from the Protein Data Bank. If you run into errors with your PDB file, ask your instructor. You should have this file from your MD run in step 1.

If you want to remove waters from your files (you may want this if you did not use the Step 1 colab), you will need a .prmtop file:
*  **.prmtop** is the topology of your protein. Toplogy means how the atoms connect together and the location of each atom. You should have this file from Step 1.

If you would like a tutorial on Python basics, please reference [this colab](https://colab.research.google.com/drive/1UVupXh23ArJp2F9vqTmkVh0JnjEaemi1?usp=sharing ). It is not necessary to do this, however if you are interested in learning more about coding it is a great resource to use!

This should take approximately 30 minutes to run.

---

**Acknowledgments**
- Colab developed by **Angela Kayll, Angelina Sardelli, and Christopher Berndsen** at James Madison University (ver 2 2025) with input and feedback from students in CHEM260 and members of the [MCC community](https://www.mdhcures.org/).

Ver 1 of this colab is available [here](https://colab.research.google.com/drive/1gsqVx570FsOm7-aDZNslvHc4Kkq2Gp1v?usp=sharing)


In [ ]:
#@title **1a. Install Conda Colab**
#@markdown Press PLAY. It will restart the kernel (session), don't worry. But do not start
#@markdown any other cells until this one finishes!
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title **1b. Install and load dependencies**
#@markdown Press PLAY **after** the above cell has completed and the session restarted.
#@markdown This will take a few minutes.
!pip install -q mdtraj
!pip install -q matplotlib
!pip install --upgrade MDAnalysis --quiet
!pip install -q pandas
!pip install -q seaborn
!pip install -q propkatraj

import pandas as pd
import mdtraj as md
import matplotlib.pyplot as pp
import MDAnalysis as mda
import os
from MDAnalysis.analysis import rms, align, diffusionmap
import seaborn as sns
from propkatraj import PropkaTraj

In [ ]:
# Load Google Drive
#@title ### **1c. Import Google Drive**
#@markdown Click PLAY to make your Google Drive accessible.
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Once both of the above have run, *then* you can begin analyzing the trajectory.

1.   Click on the file folder on the left side
2.   **Right-click**, bring up the actions menu, and create a folder. Name this folder "data".
3. Upload the .dcd file and the .pdb file. Put these files in the "data" folder. The .dcd files are often large so be patient.
4.   **Right-click**, bring up the actions menu, and create a folder. Name this folder "outputs". This is where we will put any output files or figures. **Remember to download all of these files when done.**
5.  I'll provide instructions on the basic commands and analyses, if you add more remember to save what you are doing.






---

## **1d. Loading your simulation files**

At this point, you should have all libraries and dependencies installed and all necessary input files already in your Google Drive folder. This is what you did in the last section! Now, you will upload the file that describes the structure of your protein. This file is called a .pdb file.

Below, you should provide the names of all input files and the pathway of your Google Drive folder containing them.

*   **PDB_file_name** is the name of your _nw .pdb file. Add the name where it says "xxx". Make sure there are no extra spaces in the file name!
*   **DCD_file_name** is the name of your _nw .dcd file. Add the name where it says "xxx". Make sure there are no extra spaces in the file name!
*  **Google_Drive_path** is where your simulation files are. After MyDrive/, paste the name of your folder where the "xxx" is. Make sure there are no spaces in the folder name!

When you are ready, press PLAY.

**If this code has run successfully, you should see a new folder in the directory called outputs**

In [ ]:
#@title Please provide the necessary input information below:
#@markdown Please reference the text box above for instructions regarding file name inputs and directories.
%%capture
PDB_file_name = 'xxx.pdb' #@param {type:"string"}
DCD_file_name = 'xxx.dcd' #@param {type:"string"}
file_name = PDB_file_name
Google_Drive_Path = '/content/drive/MyDrive/xxx' #@param {type:"string"}
workDir = Google_Drive_Path

# make the outputs directory
!mkdir -p $workDir/outputs

# connect to MDAnalysis
dcd =  os.path.join(workDir, str(DCD_file_name))
pdb = os.path.join(workDir, str(PDB_file_name))
u = mda.Universe(pdb, dcd)


# Analysis of the trajectory
We will use the MDAnalysis package to do the analysis and Matplotlib to plot our results. In the first step we will load the no solvent trajectory file that we made previously and use the pdb file from above to define the system.

---
## 2. RMSD analysis

For the **first box**, it will calculate your root mean square deviation (RMSD), which will indicate how much your trajectory changed from the start to the end of your simulation. You do not need to edit anything in this box unless your instructor tells you to.

For the **second box**, it will make a pretty spreadsheet of your RMSD results and save it as a CSV file. It will also show you the first five rows of data to allow you to assess the quality of the output. You do not need to edit anything in this box unless your instructor tells you to.

For the **third box**, it will plot your data as a png file. The RMSD plot gives a sense of whether the protein has reached a fixed state over the simulation, but it is not particularly good at comparing the structures overall. You do not need to edit anything in this box unless your instructor tells you to.

In [ ]:
#@title Calculate RMSD
# calculate rmsd
# RMSD = root mean square deviation, and indicates how much the trajectory changed from start to end
u.trajectory[0] # set to first frame
rmsd_analysis = rms.RMSD(u, select='backbone', groupselections=['name CA', 'protein']) # select the areas to analyze, in this case the protein backbone atoms
rmsd_analysis.run(); # run the analysis

In [ ]:
#@title Make the data frame
# make a pretty data frame using pandas
rmsd_df = pd.DataFrame(rmsd_analysis.results.rmsd[:, 1:],
                       columns=['time', 'Backbone', 'C-alphas', 'Protein'])

# save this information as a CSV file
rmsd_file = os.path.join(workDir, 'outputs', 'rmsd_df.csv')
rmsd_df.to_csv(rmsd_file)

# show the first five rows to make sure it looks right
rmsd_df.head()

In [ ]:
#@title Show the RMSD plot
# show the data in frame (you can change the axes values as needed)
fig, ax = pp.subplots()
ax.plot(rmsd_df['time']/1000, rmsd_df['Backbone'])
ax.set(xlabel = 'time', ylabel = 'RMSD')

# save the plot
rmsdpp_file = os.path.join(workDir, 'outputs', 'rmsd_plot.png')
pp.savefig(rmsdpp_file)
pp.show()



---
## 3. Root Mean Square Fluctuation plot

The code below calculates the root mean square fluctuation, effectively determining how much motion each amino acid has during the simulation. It is similar to RMSD, except instead of examining positional differences in the entire structure over time, it specifies the movement/fluctuation of each residue during a simulation.



---



In [ ]:
#@title Calculate the RMSF
# calculate the root mean square flucuation, effectively how much motion each amino acid has during the simulation
calphas = u.select_atoms("name CA") #Commonly the RMSD is computed using the aC of the protein to evaluate protein flucturation respecting the initial coordinates
rmsfer = rms.RMSF(calphas,verbose=False).run()
RMSF_table=pd.DataFrame(rmsfer.rmsf,index=calphas.resnums,columns=['RMSF']) #Saving the RMSF value for every aC residue
# save the data
rmsf_file = os.path.join(workDir, 'outputs', 'rmsf_df.csv')
RMSF_table.to_csv(rmsf_file)

# make the plot
pp.rcParams['axes.linewidth'] = 1.5
fig, ax = pp.subplots(figsize=(8,6))

pp.plot(calphas.resnums, rmsfer.rmsf,linewidth=1.5,color='k')

pp.xlabel ('Residue Cα',fontsize=16,fontweight='bold')
pp.ylabel ('RMSF (Å)',fontsize=16,fontweight='bold')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

pp.tick_params ('both',width=2,labelsize=12)
pp.grid (axis='y',alpha=0.5)
pp.tight_layout()
rmsfpp_file = os.path.join(workDir, 'outputs', 'rmsf_plot.png')
pp.savefig(rmsfpp_file)
pp.show()
# save the plot


## 4. Predict pKa values
This takes a bit of time (20+ minutes). Produces a spreadsheet of pKa at each step in the trajectory for each amino acid side chain that typically ionizes under biological conditions. The amino acids are indicated by residue number in the protein, so it may not exactly match the sequence numbering if the simulation uses a distinct numbering scheme.
The describe cell gives a statical glimpse of the trajectory, while the save the data cell saves the entire set of data for plotting using another program.

**Make sure to only press PLAY once the previous run is complete!!**

In [ ]:
#@title Predict the pKa values
pkatraj = PropkaTraj(u, select='protein', skip_failure=True)
pkatraj.run(step = 3)

In [ ]:
#@title Show the summary file
#  show the summary file
pkatraj.results.pkas.describe()

In [ ]:
#@title Save the pKa data
# save the pka data
pka_file = os.path.join(workDir, 'outputs', 'pkas_over_traj.csv')
pkatraj.results.pkas.to_csv(pka_file)

# save pkatraj.results.pkas.describe() file to csv
des_df = pkatraj.results.pkas.describe()
des_file = os.path.join(workDir, 'outputs', 'pkas_describe.csv')
des_df.to_csv(des_file)

**For this next section, you will need to change the "xxx" at the end of the line of code in the brackets to the number of the specific amino acid you would like to look at.** For example, if you want to look at the trajectory of amino acid 176, put that into the [ ]. If you have an error message show up, make sure that the amino acid number is present in your "pkas_over_traj.csv" file (you should see a column with that number). This will allow you to see differences in the amino acid that you have decided to change for your variant protein!

In [ ]:
# to see one specific amino acid over the trajectory use the code below
# change the number in the [] to be the amino acid number in the structure, remember the numbering may be different so check the numbering in the pdb file
pp.plot(pkatraj.results.pkas.index,pkatraj.results.pkas[xxx])

In [ ]:
# to see multiple columns as a boxplot
#replace xxx, yyy, zzz with the specific amino acid numbers. You can add more names by adding commas between each new number.
df = pkatraj.results.pkas[[xxx, yyy, zzz]]

# results will be in the form of a boxplot
df.boxplot()

# Congrats, you have finished Step 2 of the MD Colab! :-)

